In [1]:
proj_list =[
    'boringssl', 'c-ares',
    'freetype2', 'guetzli',
    'harfbuzz', 'libpng',
    'libssh', 'libxml2',
    'pcre', 'proj4',
    'r32', 'sqlite3',
    'vorbis', 'woff2',
    'wpantund'
]

version = 'version3'
proj_list = [
    'total_aspell', 'total_boringssl', 'total_c-ares', 'total_exiv2',
    'total_freetype2', 'total_grok', 'total_guetzli', 'total_harfbuzz',
    'total_lcms', 'total_libarchive', 'total_libexif', 'total_libhtp',
    'total_libpng', 'total_libsndfile', 'total_libssh', 'total_libxml2',
    'total_ndpi', 'total_openthread', 'total_pcre2', 'total_proj4',
    'total_re2', 'total_sqlite3', 'total_usrsctp', 'total_vorbis',
    'total_woff2', 'total_wpantund', 'total_yara', 'total_zstd'
]

In [2]:
target_project = 0

In [3]:
from sklearn.model_selection import train_test_split
import torch
import torch, gc

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import data_loader as dl
import initializer as init
import trainer
import tester
# import predictor
import model_util as mu

import os

In [4]:
gc.collect()
torch.cuda.empty_cache()

# print(torch.cuda.memory_summary(device=None, abbreviated=False))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
'''
{
    "prefix":[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 110, 102, 123, 54],
    
    "prefix-ids":["", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "TBBox_Rec_", ""],
    
    "postfix": [123, 37, 123, 72, 123, 8, 123, 85, 123, 72, 123, 8, 123, 48, 123, 8, 123, 55, 52, 91, 123, 8, 123, 37, 123, 72, 123, 8, 123, 85, 123, 72, 123, 8, 123, 57, 123, 8, 123, 55, 52, 54, 30, 54, 85, 123, 97, 123, 22, 123, 97, 123, 55, 123, 53, 99, 91, 123, 84, 91, 123, 123, 91, 123],
    
    "postfix-ids": ["to", "", "xMax", "", "bbox", "", "user", "", "xMax", "", "bbox", "", "user", "", "x", "", "to", "", "", "", "x", "", "to", "", "xMin", "", "bbox", "", "user", "", "xMin", "", "bbox", "", "user", "", "x", "", "to", "", "", "", "", "", "", "user", "", "TBBox_Rec", "", "to", "", "FT_Vector", "", "BBox_Move_To", "", "", "", "TBBox_Rec", "", "", "bbox", "FT_BBox", "", "last"],
    
    "label-type": [123],
    
    "label-prefix": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
    
    "label-postfix": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
    
    "case": 2
}
'''

'\n{\n    "prefix":[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 110, 102, 123, 54],\n    \n    "prefix-ids":["", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "TBBox_Rec_", ""],\n    \n    "postfix": [123, 37, 123, 72, 123, 8, 123, 85, 123, 72, 123, 8, 123, 48, 123, 8, 123, 55, 52, 91, 123, 8, 123, 37, 123, 72, 123, 8, 123, 85, 123, 72, 123, 8, 123, 57, 123, 8, 123, 55, 52, 54, 30, 54, 85, 123, 97, 123, 22, 123, 97, 123, 55, 123, 53, 99, 91, 123, 84, 91, 123, 123, 91, 123],\n    \n    "postfix-ids": ["to", "", "xMax", "", "bbox", "", "user", "", "xMax", "", "bbox", "", "user", "", "x", "", "to", "", "", "", "x", "", "to", "", "xMin", "", "bbox", "", "user", 

In [6]:
# get all data exept target project
prefix_np, prefix_ids_np,\
postfix_np, postfix_ids_np,\
label_np,\
label_prefix_np, label_postfix_np,\
case_np = data.getTrainData(proj_list, proj_list[target_project], version)

Getting data for "total_aspell" from "total_aspell"
Getting data for "total_aspell" from "total_boringssl"
Getting data for "total_aspell" from "total_c-ares"
Getting data for "total_aspell" from "total_exiv2"
Getting data for "total_aspell" from "total_freetype2"
Getting data for "total_aspell" from "total_grok"
Getting data for "total_aspell" from "total_guetzli"
Getting data for "total_aspell" from "total_harfbuzz"
Getting data for "total_aspell" from "total_lcms"
Getting data for "total_aspell" from "total_libarchive"
Getting data for "total_aspell" from "total_libexif"
Getting data for "total_aspell" from "total_libhtp"
Getting data for "total_aspell" from "total_libpng"
Getting data for "total_aspell" from "total_libsndfile"
Getting data for "total_aspell" from "total_libssh"
Getting data for "total_aspell" from "total_libxml2"
Getting data for "total_aspell" from "total_ndpi"
Getting data for "total_aspell" from "total_openthread"
Getting data for "total_aspell" from "total_pcre

In [7]:
# get target project data
'''
test_prefix, test_postfix,\
test_label, test_label_prefix,\
test_label_postfix, test_case = data.getTestData(proj_list[target_project])
'''

'\ntest_prefix, test_postfix,test_label, test_label_prefix,test_label_postfix, test_case = data.getTestData(proj_list[target_project])\n'

In [9]:
prefix_np, unuse_prefix,\
prefix_ids_np, unuse_prefix_ids,\
postfix_np, unuse_postfix,\
postfix_ids_np, unuse_postfix_ids,\
label_np, unuse_label,\
label_prefix_np, unuse_label_prefix,\
label_postfix_np, unuse_label_postfix,\
case_np, unuse_case = train_test_split(
    prefix_np,
    prefix_ids_np,
    postfix_np,
    postfix_ids_np,
    label_np,
    label_prefix_np,
    label_postfix_np,
    case_np,
    test_size = 0.5, random_state = 43
)

In [10]:
train_prefix, test_prefix,\
train_prefix_ids, test_prefix_ids,\
train_postfix, test_postfix,\
train_postfix_ids, test_postfix_ids,\
train_label, test_label,\
train_label_prefix, test_label_prefix,\
train_label_postfix, test_label_postfix,\
train_case, test_case = train_test_split(
    prefix_np,
    prefix_ids_np,
    postfix_np,
    postfix_ids_np,
    label_np,
    label_prefix_np,
    label_postfix_np,
    case_np,
    test_size = 0.2, random_state = 43
)

In [11]:
train_prefix, val_prefix,\
train_prefix_ids, val_prefix_ids,\
train_postfix, val_postfix,\
train_postfix_ids, val_postfix_ids,\
train_label, val_label,\
train_label_prefix, val_label_prefix,\
train_label_postfix, val_label_postfix,\
train_case, val_case = train_test_split(
    train_prefix,
    train_prefix_ids,
    train_postfix,
    train_postfix_ids,
    train_label,
    train_label_prefix,
    train_label_postfix,
    train_case,
    test_size = 0.2, random_state = 43
)

In [12]:
print('train: ', len(train_label))
print('validation: ', len(val_label))
print('test: ', len(test_label))

train:  339916
validation:  84980
test:  106224


In [13]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, val_prefix, test_prefix,
        train_prefix_ids, val_prefix_ids, test_prefix_ids,

        train_postfix, val_postfix, test_postfix,
        train_postfix_ids, val_postfix_ids, test_postfix_ids,

        train_label, val_label, test_label,

        train_label_prefix, val_label_prefix, test_label_prefix,
        train_label_postfix, val_label_postfix, test_label_postfix,

        train_case, val_case, test_case,

        batch_size=1000
    )

In [14]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [15]:
# ====================
# set parameters here
# ====================

overall_title = 'version3_'
title = overall_title + '01'
epochs = 20 

# max_len, source_code_tokens, token_choices = data.getInfo()

learning_rate = 0.001
weight_decay = 0.0

embed_dim = 100 # 100
hidden_size = 200 # 200
n_layers = 1
output_size = 1 # max(token_choices) + 1
dropout = 0.0
max_length = 64 # max_len
input_size = 154 # max(token_choices) + 1
device = device

model_name = "seq2seq"
optim_name = "Adam"
loss_fn_name = "BCE"

teacher_forcing_ratio = 0.75
threshold = torch.tensor([0.5]).to(device)

In [16]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/'+title+'/tests')

In [17]:
trainer.set_seed(42)

model, loss_fn, optimizer = init.initialize_model(
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    n_layers=n_layers,
    output_size=output_size,
    dropout=dropout,
    max_length=max_length,
    input_size=input_size,
    device=device,
    loss_fn_name=loss_fn_name
)

print(model)

MySeq2Seq(
  (prefixEncoder): Encoder(
    (embedding): Embedding(154, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
    (hidden_fc): Linear(in_features=200, out_features=100, bias=True)
    (cell_fc): Linear(in_features=200, out_features=100, bias=True)
    (dp): Dropout(p=0.0, inplace=False)
  )
  (postfixEncoder): Encoder(
    (embedding): Embedding(154, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
    (hidden_fc): Linear(in_features=200, out_features=100, bias=True)
    (cell_fc): Linear(in_features=200, out_features=100, bias=True)
    (dp): Dropout(p=0.0, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(154, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
  )
  (pointer): PointerNetwork(
    (fc1): Linear(in_features=808, out_features=400, bias=True)
    (fc2): Linear(in_features=400, out_features=200, bias=True)
    (fc3): Linear(in_features=200, out_features=1, bias=True)
    (dp1): Dropou

In [18]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    teacher_forcing_ratio=teacher_forcing_ratio,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    device=device,
    threshold=threshold
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.162741   | 94.619564  | 0.142565 | 93.87  | 577.04
   2    |   0.139419   | 95.035617  | 0.135370 | 93.41  | 530.47
   3    |   0.132131   | 95.218646  | 0.129238 | 92.89  | 528.82
   4    |   0.126915   | 95.361272  | 0.125266 | 92.82  | 528.22
   5    |   0.124669   | 95.403586  | 0.123949 | 93.37  | 528.52
   6    |   0.123150   | 95.469211  | 0.121460 | 92.97  | 528.15
   7    |   0.121071   | 95.539878  | 0.120643 | 91.06  | 526.54
   8    |   0.118852   | 95.595405  | 0.118157 | 92.57  | 527.32
   9    |   0.117620   | 95.623355  | 0.117091 | 92.65  | 531.36
  10    |   0.115873   | 95.681875  | 0.115895 | 92.03  | 528.36
  11    |   0.114498   | 95.720478  | 0.114907 | 92.51  | 528.49
  12    |   0.113445   | 95.736442  | 0.114484 | 92.14  | 529.03
  13    |   0.112211   | 95.766637  | 0.113821 | 90.8

In [19]:
mu.saveModel(overall_title, title, model)

In [20]:
model = mu.getModel(overall_title, title)
print(model)

MySeq2Seq(
  (prefixEncoder): Encoder(
    (embedding): Embedding(154, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
    (hidden_fc): Linear(in_features=200, out_features=100, bias=True)
    (cell_fc): Linear(in_features=200, out_features=100, bias=True)
    (dp): Dropout(p=0.0, inplace=False)
  )
  (postfixEncoder): Encoder(
    (embedding): Embedding(154, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
    (hidden_fc): Linear(in_features=200, out_features=100, bias=True)
    (cell_fc): Linear(in_features=200, out_features=100, bias=True)
    (dp): Dropout(p=0.0, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(154, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
  )
  (pointer): PointerNetwork(
    (fc1): Linear(in_features=808, out_features=400, bias=True)
    (fc2): Linear(in_features=400, out_features=200, bias=True)
    (fc3): Linear(in_features=200, out_features=1, bias=True)
    (dp1): Dropou

In [21]:
loss, acc, TT_acc = tester.test(
    test_dataloader=test_dataloader,
    model=model,
    loss_fn=loss_fn,
    device=device,
    fn=overall_title,
    proj_nm=title,
    threshold=threshold
)

test loss:  0.10804180332892652
test acc:  95.86014887971699
TT acc:  49.93396226415094
saved precision and recall results to file!


In [22]:
with open('../stat/'+overall_title, 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\t |\t TT acc: ' + str(TT_acc)
        # text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(0.0, 3)) + ' min\t |\t TT acc: ' + str(TT_acc)
        f.write(text)

In [23]:
mu.graphModel(train_dataloader, model, writer, device)

uploaded model graph to tensorboard!
